In [45]:
from tensorflow import keras
import tensorflow as tf
import numpy as np

In [55]:
def build_test_tfrecord(img,test_tfrecord):  # Generate TFRecord of test set
    with tf.io.TFRecordWriter(test_tfrecord)as writer:
            image = open(img, 'rb').read()

            feature = {
                'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image])),
            }

            example = tf.train.Example(features=tf.train.Features(feature=feature))
            writer.write(example.SerializeToString())

In [56]:
def _parse_example(example_string):
    feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
    }

    feature_dict = tf.io.parse_single_example(example_string, feature_description)
    feature_dict['image'] = tf.io.decode_png(feature_dict['image'], channels=3)
    feature_dict['image'] = tf.image.resize(feature_dict['image'], [224, 224]) / 255.0
    return feature_dict['image']

In [57]:
def get_test_dataset(test_tfrecord):
    raw_test_dataset = tf.data.TFRecordDataset(test_tfrecord)
    test_dataset = raw_test_dataset.map(_parse_example)

    return test_dataset

In [58]:
def data_Preprocessing(test_dataset):

    test_dataset = test_dataset.batch(32)
    test_dataset = test_dataset.prefetch(tf.data.experimental.AUTOTUNE)

    return test_dataset

In [59]:
def test(test_dataset):

    mod=keras.models.load_model('Acne and Rosacea/Layer2-0/Layer2-0skindiseases.h5')
    predIdxs = mod.predict(test_dataset)
    predIdxs = np.argmax(predIdxs, axis=1) 
    return predIdxs

In [60]:
def simulation(img_path):
    test_tfrecord = 'test.tfrecords'
    build_test_tfrecord(img_path, test_tfrecord)
    test_dataset = get_test_dataset(test_tfrecord)
    test_dataset = data_Preprocessing(test_dataset) 
    test(test_dataset)
    return test(test_dataset) 

In [61]:
imgPath = input('imge Path')
simulation(imgPath)

imge Pathtest/R (1).jpeg


array([1], dtype=int64)